In [15]:
import pandas as pd

In [16]:
data_min=pd.read_csv('annotations_df/annotations_by_min.tsv',sep='\t')
data_biya=pd.read_csv('annotations_df/annotations_by_biya.tsv',sep='\t')
data_jessie=pd.read_csv('annotations_df/annotations_by_jessie.tsv',sep='\t')
data_jinhong=pd.read_csv('annotations_df/annotations_by_jinhong.tsv',sep='\t')
data_jae=pd.read_csv('annotations_df/annotations_by_jae.tsv',sep='\t')

In [32]:
data_min

,category,author,title,type,collection,year,course
0,Optional,"[{:family=>""Reynolds"", :given=>""T.D.""}, {:fami...",Unit Operations and Processes in Environmental...,NaN,NaN,1996,58
1,Optional,"[{:family=>""Mara"", :given=>""D.""}]",Domestic Wastewater Treatment in Developing Co...,NaN,NaN,2003,58
2,Optional,"[{:family=>""Viessman"", :given=>""W."", :suffix=>...",Water Supply and Pollution Control,NaN,NaN,2005,58
3,Optional,"[{:family=>""Tchobanoglous"", :given=>""G.""}, {:f...",Wastewater Engineering: Treatment and Reuse,NaN,NaN,2003,58
4,Optional,"[{:family=>""Staff"", :given=>""M.W.H.""}]",Water Treatment: Principles and Design,NaN,NaN,2005,58
...,...,...,...,...,...,...,...
1710,Required,"[{:family=>""Spielman"", :given=>""Daniel""}]",Chapter 16: Spectral Graph Theory,book,NaN,2007,7
1711,Optional,"[{:given=>""Battiston""}, {:others=>true}]",DebtRank: Too Central to Fail? Financial Networks,NaN,NaN,2012,7
1712,Optional,"[{:given=>""Akbarpour""}, {:others=>true}]",Just a Few Seeds More: Value of Network Inform...,NaN,NaN,2018,7
1713,Optional,"[{:given=>""Shah""}, {:given=>""Zaman""}]",Rumors in a Network: Who’s the Culprit?,NaN,NaN,2011,7


In [27]:
def get_agreement(df):
    """Return the agreement score of the dataframe

    Parameters
    ----------
    df : pandas dataframe
        the dataframe for agreement score calculation

    Returns
    -------
    double
        the agreement score of the dataframe
    """
    
    total = df.sum().sum()

    A_0 = (df.loc['Required']['Required'] + df.loc['Optional']['Optional']) / total
    A_e = 0

    col_num = df.shape[0]

    for i in range(col_num):
        A_e += ((df.iloc[i,:].sum() + df.iloc[:,i].sum()) / (total*2))**2
    agreement = (A_0 - A_e)/ (1-A_e)
    
    return agreement

In [22]:
def get_dataframe_for_calculation(df_1, df_2):
    """Return the dataframe for agreement calculation

    Parameters
    ----------
    df_1 : pandas dataframe of annotator 1
        the dataframe for produced by annotator 1
    df_2 : pandas dataframe of annotator 2
        the dataframe for produced by annotator 2

    Returns
    -------
    pandas dataframe
        the dataframe for agreement score calculation
    """
    
    # inner join
    merged_df = df_1.merge(df_2, on=['title', 'course'])
    merged_df["X_Y"] = tuple(zip(merged_df["category_x"], merged_df["category_y"]))
    
    # initialize dataframe
    df_t = pd.DataFrame({'Required': [0, 0, 0],
                         'Optional': [0, 0, 0],
                         'UNK': [0, 0, 0]},
                         index=['Required', 'Optional', 'UNK'])
    
    # count RR/RO/OR/OO
    dic = merged_df["X_Y"].value_counts().to_dict()
    for key, val in dic.items():
        df_t.loc[key[0]][key[1]] = val
    
    # left join for UNK
    left_df = df_1.merge(df_2, on='title', how='left')
    left_df = df_1.merge(df_2.drop_duplicates(), on='title',how='left', indicator=True)
    left_df = left_df[left_df['_merge'] == 'left_only']
    
    dic_left = left_df["category_x"].value_counts().to_dict()
    for key, val in dic_left.items():
        df_t.loc[key]['UNK'] = val
    
    # right join for UNK
    right_df = df_1.merge(df_2.drop_duplicates(), on='title',how='right', indicator=True)
    right_df = right_df[right_df['_merge'] == 'right_only']
    
    dic_right = right_df["category_y"].value_counts().to_dict()
    for key, val in dic_right.items():
        df_t.loc['UNK'][key] = val
        
    return df_t

In [30]:
get_dataframe_for_calculation(data_jae, data_biya)

,Required,Optional,UNK
Required,1913,107,133
Optional,12,915,52
UNK,131,108,0


In [31]:
get_agreement(get_dataframe_for_calculation(data_jae, data_biya))

0.683192401822387